In [ ]:
import pandas as pd
!pip3 install transformers
!pip3 install simpletransformers
!pip3 install logging
from simpletransformers.language_modeling import LanguageModelingModel
import logging
from simpletransformers.language_modeling import LanguageModelingModel
import logging
from simpletransformers.language_modeling import LanguageModelingModel
from simpletransformers.language_modeling import LanguageModelingModel
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch
import logging

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from simpletransformers.language_modeling import LanguageModelingModel
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch
import logging
import pandas as pd
from sklearn.model_selection import train_test_split

values = list(range(0, 40))
keys = [' Allergy / Immunology', ' Autopsy', ' Bariatrics', ' Cardiovascular / Pulmonary', ' Chiropractic', ' Consult - History and Phy.', ' Cosmetic / Plastic Surgery', ' Dentistry', ' Dermatology', ' Diets and Nutritions', ' Discharge Summary', ' Emergency Room Reports', ' Endocrinology', ' ENT - Otolaryngology', ' Gastroenterology', ' General Medicine', ' Hematology - Oncology', ' Hospice - Palliative Care', ' IME-QME-Work Comp etc.', ' Lab Medicine - Pathology', ' Letters', ' Nephrology', ' Neurology', ' Neurosurgery', ' Obstetrics / Gynecology', ' Office Notes', ' Ophthalmology', ' Orthopedic', ' Pain Management', ' Pediatrics - Neonatal', ' Physical Medicine - Rehab', ' Podiatry', ' Psychiatry / Psychology', ' Radiology', ' Rheumatology', ' Sleep Medicine', ' SOAP / Chart / Progress Notes', ' Speech - Language', ' Surgery', ' Urology']

#hashmap1 = {keys[i]: values[i] for i in range(len(keys))}
#print(hashmap1)
# perform stratified 30/70 split on the data
df= pd.read_csv('input.csv')
train_df, eval_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df["label"])

print(train_df.head())
print("")
print(eval_df.head())
# print the number of samples in each split
print(f'Train data samples: {len(train_df)}')
print(f'Test data samples: {len(eval_df)}')
 

train_df.columns = ["text", "labels"]
eval_df.columns = ["text", "labels"]


df = pd.read_csv('input.csv')
df.head()
df2 = df.filter(['transcript_sentences', 'label'], axis=1)

# for index, row in df2.iterrows():
#   # print(row['medical_specialty'])
#   row['medical_specialty']= hashmap1[row['medical_specialty']]

df2.columns = ["text", "labels"]
#df2.head()

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_file = df2

model_args = ClassificationArgs(num_train_epochs=6)

model = ClassificationModel("electra", "enelpi/med-electra-small-discriminator", num_labels = 40, args=model_args, use_cuda = True)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
  
print(result)


                                     transcript_sentences  label
179330  Doppler study reveals mild-to-moderate mitral ...      3
101738  She was last seen by us in clinic in March 200...     22
179361  Insertion of a right subclavian vein triple lu...      3
14506   The calculated systemic flow was normal and pu...     38
32605   The bone flap was then replaced and sutured wi...     38

                                     transcript_sentences  label
131194  History of perforated sigmoid diverticuli with...     14
185401  Right coronary system was evaluated in several...      3
141341                       VITAL SIGNS:  He is afebrile     11
120012  OBJECTIVE:,General:  The patient is a very ple...     15
138238                 No additional bleeding was evident     13
Train data samples: 135996
Test data samples: 58284


Some weights of the model checkpoint at enelpi/med-electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at enelpi/med-electra-small-discriminator and are newly initialized: ['cl

  0%|          | 0/135996 [00:00<?, ?it/s]

KeyboardInterrupt: ignored